In [40]:
# import necessary pacakages
# strange issue: keep the import order to prevent matplotlib error
#  import matplotlib -> librosa -> pyplot -> librosa.display
import sys
import numpy as np
import matplotlib
import librosa
from matplotlib import pyplot as plt
import librosa.display

#from scipy.io import wavfile
from scipy import signal
from scipy.fft import fftshift

import os 

# Classification of voiced, unvoiced, and silent
voiced = ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'axr', 'ay', 'eh', 'er', 'ey', 'ih',\
        'ix', 'iy', 'ow', 'oy', 'uh', 'uw', 'ux', 'b', 'd', 'dh', 'el', 'em', \
        'en', 'g', 'jh', 'l', 'm', 'n', 'ng', 'nx', 'r', 'v', 'w', 'wh', 'y', \
        'z', 'zh', 'eng', 'hv', 'bcl', 'dcl', 'gcl']
unvoiced = ['ch', 'dx', 'f', 'hh', 'k', 'p', 'q', 's', 'sh', 't', 'th', 'ax-h']
silence = ['kcl', 'pcl', 'tcl', 'pau', 'epi', 'h#']

### Dividing the dataset into train and validation sets
# The TIMIT dataset is divided into 8 dialect regions, each with 50 speakers.
# We will use 80% of the speakers for training and 20% for validation.
# The train and validation sets are saved in the lists train and val respectively.
# Each element of the list is the path to the speaker's folder.
 
timit_path = './audio/timit_wav/'
dialects = os.listdir(timit_path+'train')
dialects = sorted(dialects)[1:]

train = []
val = []

for dialect in dialects:
    speakers = os.listdir(timit_path+'train/'+dialect)
    # if '.DS_Store' in speakers: speakers.remove('.DS_Store')
    train_speakers = speakers[:int(len(speakers)*0.8)]
    val_speakers = speakers[int(len(speakers)*0.8):]
    for i in range(len(train_speakers)):
        train.append(timit_path+'train/'+dialect+'/'+train_speakers[i])
    for i in range(len(val_speakers)):
        val.append(timit_path+'train/'+dialect+'/'+val_speakers[i])

print(len(train), len(val))

### Reading in the test set
# The test set is stored in the list test.
# Each element of the list is the path to the speaker's folder.
# The test set is divided into 8 dialect regions, each with 24 speakers.

test = []

dialects_test = os.listdir(timit_path+'test')
dialects_test = sorted(dialects_test)
# if '.DS_Store' in dialects_test: dialects_test.remove('.DS_Store')

for dialect in dialects_test:
    speakers = os.listdir(timit_path+'test/'+dialect)
    # if '.DS_Store' in speakers: speakers.remove('.DS_Store')
    for i in range(len(speakers)):
        test.append(timit_path+'test/'+dialect+'/'+speakers[i])

print(len(test))



336 88
168


In [41]:
print(train)

['./audio/timit_wav/train/dr2/fmmh0', './audio/timit_wav/train/dr2/mjde0', './audio/timit_wav/train/dr2/msat0', './audio/timit_wav/train/dr2/mprb0', './audio/timit_wav/train/dr2/fjkl0', './audio/timit_wav/train/dr2/mdps0', './audio/timit_wav/train/dr2/marc0', './audio/timit_wav/train/dr2/mrjt0', './audio/timit_wav/train/dr2/mkaj0', './audio/timit_wav/train/dr2/fsrh0', './audio/timit_wav/train/dr2/mjae0', './audio/timit_wav/train/dr2/mefg0', './audio/timit_wav/train/dr2/mdmt0', './audio/timit_wav/train/dr2/mtbc0', './audio/timit_wav/train/dr2/fdxw0', './audio/timit_wav/train/dr2/flmc0', './audio/timit_wav/train/dr2/mjhi0', './audio/timit_wav/train/dr2/mmds0', './audio/timit_wav/train/dr2/mrms0', './audio/timit_wav/train/dr2/mjbg0', './audio/timit_wav/train/dr2/fcaj0', './audio/timit_wav/train/dr2/mrjm0', './audio/timit_wav/train/dr2/mhrm0', './audio/timit_wav/train/dr2/fcyl0', './audio/timit_wav/train/dr2/mrjh0', './audio/timit_wav/train/dr2/mzmb0', './audio/timit_wav/train/dr2/mrcw0', 

In [42]:
speech_train_SA = [] # list형태로 파일을 받음
speech_train_SX = [] # list형태로 파일을 받음
speech_train_groundtruth_SA = []
speech_train_groundtruth_SX = []
# train 파일을 SA와 SX로 분류하여서 불러옴
for i in range(len(train)):
    train_files = os.listdir(train[i])
    
    for j in range(len(train_files)):
        if train_files[j][-4:]==".wav":
            
            tmp, Fs = librosa.load(train[i]+"/"+train_files[j], sr=None, mono=True)
            # 정답데이터를 받아오는 과정
            answer = open(train[i]+"/"+train_files[j][:-4] + ".phn", 'r')
            train_answer = answer.readlines()
            answer.close()
            
            # 정답 matrix 생성
            # zeros : matrix size를 input으로 받아서 같은 사이즈의 zero-matrix를 생성
            # zeros_like : matrix를 input으로 받아서 같은 사이즈의 zero-matrix를 생성
            tmp2 = np.zeros_like(tmp)
            # print(len(tmp), len(tmp2))


            for k in range(len(train_answer)):
                start, end, det = train_answer[k][:-1].split()
                start = int(start)
                end = int(end)
                # print(start, end, det)

                # silence : 0 / voiced : 1 / unvoiced : 2  << 라이브러리에 맞게끔 분류함.
                if det in silence :
                    tmp2[start:end] = 0
                if det in voiced :
                    tmp2[start:end] = 1
                if det in unvoiced :
                    tmp2[start:end] = 2

            if train_files[j][:2]=="sa":
                speech_train_SA.append(tmp)
                speech_train_groundtruth_SA.append(tmp2)
            if train_files[j][:2]=="sx":
                speech_train_SX.append(tmp)
                speech_train_groundtruth_SX.append(tmp2)

speech_val_SA = [] # list형태로 파일을 받음
speech_val_SX = [] # list형태로 파일을 받음
speech_val_groundtruth_SA = []
speech_val_groundtruth_SX = []
tmp_array_val=[]
# validation 파일을 SA와 SX로 분류하여서 불러옴
for i in range(len(val)):
    val_files = os.listdir(val[i])
    
    for j in range(len(val_files)):
        if val_files[j][-4:]==".wav":
            tmp, Fs = librosa.load(val[i]+"/"+val_files[j], sr=None, mono=True)
            # 정답데이터를 받아오는 과정
            answer = open(val[i]+"/"+val_files[j][:-4] + ".phn", 'r')
            val_answer = answer.readlines()
            answer.close()
            
            # 정답 matrix 생성
            # zeros : matrix size를 input으로 받아서 같은 사이즈의 zero-matrix를 생성
            # zeros_like : matrix를 input으로 받아서 같은 사이즈의 zero-matrix를 생성
            tmp2 = np.zeros_like(tmp)
            # print(len(tmp), len(tmp2))


            for k in range(len(val_answer)):
                start, end, det = val_answer[k][:-1].split() # label에서 \n을 제외하고 [:-1]로 설정함.
                start = int(start) # start 값이 string형이기에 int형으로 casting해줌.
                end = int(end) # end 값이 string형이기에 int형으로 casting해줌.
                # print(start, end, det)

                # silence : 0 / voiced : 1 / unvoiced : 2  << 라이브러리에 맞게끔 분류함.
                if det in silence :
                    tmp2[start:end] = 0
                if det in voiced :
                    tmp2[start:end] = 1
                if det in unvoiced :
                    tmp2[start:end] = 2

            if val_files[j][:2]=="sa":
                tmp_array_val.append(val[i]+"/"+val_files[j])
                speech_val_SA.append(tmp)
                speech_val_groundtruth_SA.append(tmp2)
            if val_files[j][:2]=="sx":
                speech_val_SX.append(tmp)
                speech_val_groundtruth_SX.append(tmp2)
    
speech_test_SA = [] # list형태로 파일을 받음
speech_test_SX = [] # list형태로 파일을 받음
speech_test_groundtruth_SA = []
speech_test_groundtruth_SX = []
tmp_array_test=[]

# test 파일을 SA와 SX로 분류하여서 불러옴
for i in range(len(test)):
    test_files = os.listdir(test[i])
    
    for j in range(len(test_files)):
        if test_files[j][-4:]==".wav":
            tmp, Fs = librosa.load(test[i]+"/"+test_files[j], sr=None, mono=True)
            # 정답데이터를 받아오는 과정
            answer = open(test[i]+"/"+test_files[j][:-4] + ".phn", 'r')
            test_answer = answer.readlines()
            answer.close()
            
            # 정답 matrix 생성
            # zeros : matrix size를 input으로 받아서 같은 사이즈의 zero-matrix를 생성
            # zeros_like : matrix를 input으로 받아서 같은 사이즈의 zero-matrix를 생성
            tmp2 = np.zeros_like(tmp)
            # print(len(tmp), len(tmp2))


            for k in range(len(test_answer)):
                start, end, det = test_answer[k][:-1].split()
                start = int(start)
                end = int(end)
                # print(start, end, det)

                # silence : 0 / voiced : 1 / unvoiced : 2  << 라이브러리에 맞게끔 분류함.
                if det in silence :
                    tmp2[start:end] = 0
                if det in voiced :
                    tmp2[start:end] = 1
                if det in unvoiced :
                    tmp2[start:end] = 2
            
            if test_files[j][:2]=="sa":
                tmp_array_test.append(test[i]+"/"+test_files[j])
                speech_test_SA.append(tmp)
                speech_test_groundtruth_SA.append(tmp2)
            if test_files[j][:2]=="sx":
                speech_test_SX.append(tmp)
                speech_test_groundtruth_SX.append(tmp2)


In [43]:
print(tmp_array_test[0])
print(speech_test_SA[0][39237: 40928], speech_test_groundtruth_SA[0][39237: 40928])

./audio/timit_wav/test/dr1/mdab0/sa1.wav
[-0.00720215 -0.00708008 -0.00668335 ... -0.009552   -0.00686646
 -0.00033569] [1. 1. 1. ... 1. 1. 1.]


In [44]:
def V_dot(speech, Fs, Ts, Tf):
    # sample_length = len(speechest_array[0,:])
    Ns = int(Fs*Ts)
    Nf = int(Fs*Tf)
    # NFFT = int(2**(np.ceil(np.log2(Nf))))
    # hNo = NFFT//2+1
    Nt = len(speech[:])
    nframes = int((Nt-Nf)//Ns + 1)

    tmp_list_wiener = np.zeros(nframes)
    v_n = np.zeros(nframes)

    # for loop   
    tmp_list_wiener = np.array([np.sqrt(np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf) for j in range(nframes)]) # list comression type

    thresholding = (tmp_list_wiener[:10].mean() + tmp_list_wiener[10:].mean()) / 2
        
    v_n[:] = tmp_list_wiener > thresholding
    thresholding_2nd = (tmp_list_wiener[v_n[:]==0].mean() + tmp_list_wiener[v_n[:]==1].mean()) / 2

    v_n[:] = tmp_list_wiener > thresholding_2nd
        
    thresholding_3rd = (tmp_list_wiener[v_n[:]==0].mean() + tmp_list_wiener[v_n[:]==1].mean()) / 2
    v_n[:] = tmp_list_wiener > thresholding_3rd
    print(thresholding , thresholding_2nd, thresholding_3rd)
    
                
    return v_n

In [45]:
Ts = 0.01
Tf = 0.02

In [46]:
V_dot(speech_train_SA[0], Fs, Ts, Tf)


0.005503250941083789 0.009534124825075171 0.01202389125169841


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [47]:
def V_UV_auto_correlation(speech, Fs, Ts, Tf):
    # sample_length = len(speechest_array[0,:])
    Ns = int(Fs*Ts)
    Nf = int(Fs*Tf)
    # NFFT = int(2**(np.ceil(np.log2(Nf))))
    # hNo = NFFT//2+1
    Nt = len(speech[:])
    nframes = int((Nt-Nf)//Ns + 1)

    tmp_list_wiener = np.zeros(nframes)
    a = np.zeros(2)
    v_n = np.zeros(nframes)

    # for loop : 한 프레임마다 에너지를 계산함.  
    tmp_list_wiener = np.array([np.sqrt(np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf) \
                                for j in range(nframes)]) # list comression type

    thresholding = (tmp_list_wiener[:10].mean() + tmp_list_wiener[10:].mean()) / 2
        
    v_n[:] = tmp_list_wiener > thresholding
    thresholding_2nd = (tmp_list_wiener[v_n[:]==0].mean() + tmp_list_wiener[v_n[:]==1].mean()) / 2

    v_n[:] = tmp_list_wiener > thresholding_2nd
        
    thresholding_3rd = (tmp_list_wiener[v_n[:]==0].mean() + tmp_list_wiener[v_n[:]==1].mean()) / 2
    v_n[:] = tmp_list_wiener > thresholding_3rd
    
    threshold_ac = 0.5
    tmp = np.zeros(nframes)
    for j in range(nframes):
        if v_n[j] == 1:
            a[0] = np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf
            # print(a[0]) 
            a[1] = np.sum((speech[j*Ns:j*Ns+Nf])*(speech[j*Ns + 1:j*Ns+Nf + 1]))/Nf
            
            # print(a[1])
            tmp[j] = a[1]/a[0]
            # print(a[1]/a[0])
            if a[1]/a[0] > threshold_ac :
                v_n[j] = 1
                
            if a[1]/a[0] < threshold_ac :
                v_n[j] = 2

    threshold_ac_by_mean = tmp.mean()
    for j in range(nframes):
        if v_n[j] == 1:
            a[0] = np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf 
            a[1] = np.sum((speech[j*Ns + 1:j*Ns+Nf])**2)/Nf
            
            if a[1]/a[0] > threshold_ac_by_mean :
                v_n[j] = 1
            if a[1]/a[0] < threshold_ac_by_mean :
                v_n[j] = 2
            
    # to determine vocied and unvoiced speech
    return v_n

In [48]:
# print(len(tmp_array_val))
# check = 170
# print(tmp_array_val[check])

In [49]:
v_uv = V_UV_auto_correlation(speech_val_SA[100], Fs, Ts, Tf)

In [50]:
len(v_uv)

382

In [51]:
# print(v_uv[:]==1) # voiced 신호만 보기 위한 True/False Boolean 값
print(len(v_uv[v_uv[:]==1])) # voiced 신호 갯수
# print(v_uv[:]==2) # unvoiced 신호만 보기 위한 True/False Boolean 값
print(len(v_uv[v_uv[:]==2])) # unvoiced 신호 갯수
# print(v_uv[:]==0) # silence 신호만 보기 위한 True/False Boolean 값
print(len(v_uv[v_uv[:]==0])) # silence 신호 갯수

107
6
269


In [145]:
def V_UV_auto_correlation2(speech, Fs, Ts, Tf):
    # sample_length = len(speechest_array[0,:])
    Ns = int(Fs*Ts)
    Nf = int(Fs*Tf)
    # NFFT = int(2**(np.ceil(np.log2(Nf))))
    # hNo = NFFT//2+1
    Nt = len(speech[:])
    nframes = int((Nt-Nf)//Ns + 1)

    tmp_list_wiener = np.zeros(nframes)
    a = np.zeros(2)
    v_n = np.zeros(nframes)

    # for loop : 한 프레임마다 에너지를 계산함.  
    tmp_list_wiener = np.array([np.sqrt(np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf) \
                                for j in range(nframes)]) # list comression type

    thresholding = (tmp_list_wiener[:10].mean() + tmp_list_wiener[10:].mean()) / 2
        
    v_n[:] = tmp_list_wiener > thresholding
    thresholding_2nd = (tmp_list_wiener[v_n[:]==0].mean() + tmp_list_wiener[v_n[:]==1].mean()) / 2

    v_n[:] = tmp_list_wiener > thresholding_2nd
        
    thresholding_3rd = (tmp_list_wiener[v_n[:]==0].mean() + tmp_list_wiener[v_n[:]==1].mean()) / 2
    v_n[:] = tmp_list_wiener > thresholding_3rd
    
    threshold_ac = 0.5
    tmp = np.zeros(nframes)
    for j in range(nframes):
        if v_n[j] == 1:
            a[0] = np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf
            # print(a[0]) 
            a[1] = np.sum((speech[j*Ns:j*Ns+Nf])*(speech[j*Ns + 1:j*Ns+Nf + 1]))/Nf
            
            # print(a[1])
            tmp[j] = a[1]/a[0]
            # print(a[1]/a[0])
            if a[1]/a[0] > threshold_ac :
                v_n[j] = 1
                
            if a[1]/a[0] < threshold_ac :
                v_n[j] = 2

    threshold_ac_by_mean = tmp.mean()
    for j in range(nframes):
        if v_n[j] == 1:
            a[0] = np.sum((speech[j*Ns:j*Ns+Nf])**2)/Nf 
            a[1] = np.sum((speech[j*Ns + 1:j*Ns+Nf])**2)/Nf
            
            if a[1]/a[0] > threshold_ac_by_mean :
                v_n[j] = 1
            if a[1]/a[0] < threshold_ac_by_mean :
                v_n[j] = 2
    
    v_uv = np.zeros_like(speech)
    for k in range(nframes):
            v_uv[k*Ns + 1:k*Ns+Nf] = v_n[k]
            
    # to determine vocied and unvoiced speech
    return v_uv

In [146]:
# multiple validation files autocorrelation
v_uv_train_SA = []
for i in range(len(speech_train_SA)):
    v_uv2 = V_UV_auto_correlation2(speech_train_SA[i], Fs, Ts, Tf)
    v_uv_train_SA.append(v_uv2)

v_uv_train_SX = []
for i in range(len(speech_val_SX)):
    v_uv2 = V_UV_auto_correlation2(speech_train_SX[i], Fs, Ts, Tf)
    v_uv_train_SX.append(v_uv2)
 

In [147]:
# multiple validation files autocorrelation
v_uv_val_SA = []
for i in range(len(speech_val_SA)):
    v_uv3 = V_UV_auto_correlation2(speech_val_SA[i], Fs, Ts, Tf)
    v_uv_val_SA.append(v_uv3)

v_uv_val_SX = []
for i in range(len(speech_val_SX)):
    v_uv3 = V_UV_auto_correlation2(speech_val_SX[i], Fs, Ts, Tf)
    v_uv_val_SX.append(v_uv3)
 

In [148]:
print(speech_test_groundtruth_SA[0][24000:24100])

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]


In [151]:
# RNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN, Dropout, Activation, Embedding, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import tensorflow as tf

# https://stackoverflow.com/questions/54416322/expected-ndim-3-found-ndim-2
x_train = np.array(v_uv_train_SA[0], dtype=float)
# print(x_train)
# x_train = x_train.reshape(-1, 1, len(v_uv_train_SA[0]))

y_train = np.array(speech_train_groundtruth_SA[0],dtype=float)
# y_train = y_train.reshape(-1, 1, len(speech_train_groundtruth_SA[0]))
# y_train = to_categorical(y_train)
print(x_train.shape)
print(y_train.shape)

x_val, y_val = np.array(v_uv_val_SA[0],dtype=float), np.array(speech_val_SA[0],dtype=float)
x_val = x_val.reshape(-1,1,len(x_val))
y_val = y_val.reshape(-1,1,len(y_val))
y_val = to_categorical(y_val)

# definite the LSTM models
model = Sequential()
model.add(LSTM(250, input_shape=(1, len(v_uv_train_SA[0])), return_sequences=True))
# model.add(LSTM(250, return_sequences=True))
# model.add(LSTM(250))
model.add(Dense(100, activation='relu'))
# model.add(Dense(50, activation='relu'))
model.add(Dense(len(v_uv_train_SA[0]), input_shape=(1, len(v_uv_train_SA[0])), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

# model ift
# model.fit(v_uv_train_SA[0], speech_train_groundtruth_SA[0], epochs=100, batch_size=100, validation_data=(v_uv_val_SA[0], speech_val_SA[0]))
model.fit(x_train, y_train, epochs=100, batch_size=100,validation_data=(x_val, y_val))


# for i in range(len(v_uv_train_SA)):
#     model.fit(v_uv_train_SA, speech_train_groundtruth_SA, epochs=100, batch_size=100)

# for i in range(len(v_uv_val_SA)):
#     model.fit(validation_data=(v_uv_val_SA, speech_val_SA))
# model2.fit(x_train, y_train, epochs=100, batch_size=100, validation_data=(v_uv_val_SA, speech_val_SA))

[0. 0. 0. ... 0. 0. 0.]
(1, 1, 51815)
(1, 1, 51815)
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 579301.8750 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 198ms/step - loss: 578936.5000 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 197ms/step - loss: 578386.0000 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 200ms/step - loss: 577728.3750 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 194ms/step - loss: 576999.2500 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 186ms/step - loss: 576224.8750 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 189ms/step - loss: 575429.2500 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 195ms/step - loss: 574635.7500 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 198ms/step - loss: 573865.6875 - 

In [ ]:
# # RNN by pytorch
# import torch.nn as nn

# class SpeechRNN(nn.module):
#     def __init__(self, input_size, hidden_size, output_size, batch_size):
#         super(SpeechRNN, self).__init__()
#         self.batch_size = batch_size
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.rnn = nn.RNN(input_size,hidden_size)
#         self.layer = nn.Linear(hidden_size, self.output_size)
    
#     def forward(self, speech, prints=False):
